In [5]:
from utils import SystemMonitor

monitor = SystemMonitor()

# **ResNet50**

In [6]:
from SS_ResNet50 import SS_ResNet50

ss_resnet50 = SS_ResNet50()

monitor.start(interval=0.5)
ss_resnet50.predict()
monitor.stop()

monitor.compute_and_print_average()

2023-10-26 18:33:31.549403: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-26 18:33:35.395147: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-26 18:33:35.417682: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-26 18:33:44.886345: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


TypeError: 'module' object is not callable